In [2]:
# ====================================
# openai tokens
# ====================================

import os
import glob
import codecs
import pickle
import re
import textwrap
from collections import namedtuple

import openai
import faiss
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, MarkdownTextSplitter
from langchain.vectorstores import FAISS
from pymongo import MongoClient
from kipp.utils import setup_logger

from sys import path

path.append("/opt/configs/ramjet")
import prd

# ----------------------------------------------
# Azure
# ----------------------------------------------
# os.environ['OPENAI_API_TYPE'] = "azure"
# os.environ['OPENAI_API_VERSION'] = prd.OPENAI_AZURE_VERSION
# os.environ['OPENAI_API_BASE'] = prd.OPENAI_AZURE_API
# os.environ['OPENAI_API_KEY'] = prd.OPENAI_AZURE_TOKEN

# openai.api_type = os.environ['OPENAI_API_TYPE']
# openai.api_version = os.environ['OPENAI_API_VERSION']
# openai.api_base = os.environ['OPENAI_API_BASE']
# openai.api_key = os.environ['OPENAI_API_KEY']

# azure_embeddings_deploymentid = prd.OPENAI_AZURE_DEPLOYMENTS["embeddings"].deployment_id
# azure_gpt_deploymentid = prd.OPENAI_AZURE_DEPLOYMENTS["chat"].deployment_id
# ----------------------------------------------

# ----------------------------------------------
# OpenAI
# ----------------------------------------------
os.environ["OPENAI_API_KEY"] = prd.OPENAI_TOKEN_BBT

openai.api_key = os.environ['OPENAI_API_KEY']
# ----------------------------------------------

Index = namedtuple("index", ["store", "scaned_files"])


def pretty_print(text: str) -> str:
    text = text.strip()
    return textwrap.fill(text, width=60, subsequent_indent="    ")


# =============================
# 定义文件路径
# =============================

name = "bbt-xego"
logger = setup_logger(name)

index_dirpath = "/home/laisky/data/langchain/index-azure"
pdf_dirpath = f"/home/laisky/data/langchain/pdf/{name}"

for path in [index_dirpath, pdf_dirpath]:
    try:
        os.mkdir(path)
    except FileExistsError:
        pass

In [5]:
# ==============================================================
# prepare pdf documents docs.index & docs.store
#
# https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/pdf.html#retain-elements
#
# 通用的函数定义
# ==============================================================

from urllib.parse import quote

from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, MarkdownTextSplitter
from ramjet.tasks.gptchat.embedding.embeddings import reset_eof_of_pdf

text_splitter = CharacterTextSplitter(chunk_size=500, separator="\n")
markdown_splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=50)

N_BACTCH_FILES = 5


def is_file_scaned(index: Index, fpath):
    return os.path.split(fpath)[1] in index.scaned_files


def embedding_pdfs(index: Index, fpaths, url, replace_by_url):
    i = 0
    docs = []
    metadatas = []
    for fpath in fpaths:
        fname = os.path.split(fpath)[1]
        if is_file_scaned(index, fname):
            continue

        try:
            reset_eof_of_pdf(fpath)
            loader = PyPDFLoader(fpath)
            for page, data in enumerate(loader.load_and_split()):
                splits = text_splitter.split_text(data.page_content)
                docs.extend(splits)
                for ichunk, _ in enumerate(splits):
                    fnameurl = quote(fpath.removeprefix(replace_by_url), safe="")
                    furl = url + fnameurl
                    metadatas.append({"source": f"{furl}#page={page+1}"})
        except Exception:
            logger.exception(f"skip file {fpath}")
            continue

        index.scaned_files.add(fname)
        print(f"scaned {fpath}")
        i += 1
        if i > N_BACTCH_FILES:
            break

    if i != 0:
        index.store.add_texts(docs, metadatas=metadatas)

    return i


def embedding_markdowns(index: Index, fpaths, url, replace_by_url):
    i = 0
    docs = []
    metadatas = []
    for fpath in fpaths:
        fname = os.path.split(fpath)[1]
        if is_file_scaned(index, fpath):
            continue

        with codecs.open(fpath, "rb", "utf8") as fp:
            docus = markdown_splitter.create_documents([fp.read()])
            for ichunk, docu in enumerate(docus):
                docs.append(docu.page_content)
                title = quote(docu.page_content.strip().split("\n", maxsplit=1)[0])
                if url:
                    fnameurl = quote(fpath.removeprefix(replace_by_url), safe="")
                    furl = url + fnameurl
                    metadatas.append({"source": f"{furl}#{title}"})
                else:
                    metadatas.append({"source": f"{fname}#{title}"})
                    
        index.scaned_files.add(fname)
        print(f"scaned {fpath}")
        i += 1
        if i > N_BACTCH_FILES:
            break

    if i != 0:
        index.store.add_texts(docs, metadatas=metadatas)

    return i


def load_store(dirpath, name) -> Index:
    """
    Args:
        dirpath: dirpath to store index files
        name: project/file name
    """
    if os.environ.get("OPENAI_API_TYPE") == "azure":
        azure_embeddings_deploymentid = prd.OPENAI_AZURE_DEPLOYMENTS[
            "embeddings"
        ].deployment_id
        # azure_gpt_deploymentid = prd.OPENAI_AZURE_DEPLOYMENTS["chat"].deployment_id

        embedding_model = OpenAIEmbeddings(
            client=None,
            model="text-embedding-ada-002",
            deployment=azure_embeddings_deploymentid,
        )
    else:
        embedding_model = OpenAIEmbeddings(
            client=None,
            model="text-embedding-ada-002",
        )
    
    index = faiss.read_index(f"{os.path.join(dirpath, name)}.index")
    with open(f"{os.path.join(dirpath, name)}.store", "rb") as f:
        store = pickle.load(f)
    store.index = index

    with open(f"{os.path.join(dirpath, name)}.scanedfile", "rb") as f:
        scaned_files = pickle.load(f)
        
    # compatable with azure/openai embeddings
    store.embedding_function = embedding_model.embed_query

    return Index(
        store=store,
        scaned_files=scaned_files,
    )


def new_store() -> Index:
    if os.environ.get("OPENAI_API_TYPE") == "azure":
        embedding_model = OpenAIEmbeddings(
            client=None,
            model="text-embedding-ada-002",
            deployment=azure_embeddings_deploymentid,
        )
    else:
        embedding_model = OpenAIEmbeddings(
            client=None,
            model="text-embedding-ada-002",
        )
        
    store = FAISS.from_texts(["world"], embedding_model, metadatas=[{"source": "hello"}])
    return Index(
        store=store,
        scaned_files=set([]),
    )


def save_store(index: Index, dirpath, name):
    store_index = index.store.index
    fpath_prefix = os.path.join(dirpath, name)
    print(f"save store to {fpath_prefix}")
    faiss.write_index(store_index, f"{fpath_prefix}.index")
    index.store.index = None
    with open(f"{fpath_prefix}.store", "wb") as f:
        pickle.dump(index.store, f)
    index.store.index = store_index

    with open(f"{fpath_prefix}.scanedfile", "wb") as f:
        pickle.dump(index.scaned_files, f)


In [8]:
# ====================================
# 生成 Basebit XEGO Doc index
# ====================================

import os
import glob
import codecs
import pickle

import faiss
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, MarkdownTextSplitter
from langchain.vectorstores import FAISS
from pymongo import MongoClient
from ramjet.tasks.gptchat.embedding.embeddings import markdown_splitter


markdown_splitter = MarkdownTextSplitter(chunk_size=500)


def scan_xego_docs_in_mongo(index: Index) -> FAISS:
    docs = []
    metadatas = []
    dbconn = MongoClient(prd.OPENAI_EMBEDDING_QA["bbt-xego"]["mongo_source"])
    cursor = dbconn["basebit"]["docus"].find().max_time_ms(0).batch_size(50)
    i = 0
    for doc in cursor:
        i += 1
        logger.info(f'processing {doc["url"]} ...')
        splits = markdown_splitter.split_text(doc["text"])
        docs.extend(splits)
        metadatas.extend([{"source": doc["url"]}] * len(splits))
        
    dbconn.close()
    index.store.add_texts(docs, metadatas=metadatas)
    return i


def main():
    index = new_store()
#     index = load_store(
#         dirpath="/home/laisky/data/langchain/index",
#         name="bbt-xego",
#     )
    n = scan_xego_docs_in_mongo(index)
    save_store(
        index=index, 
        dirpath=index_dirpath, 
        name=name,
    )
    

main()

[2023-08-24 03:31:10,431 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/2023/02/research/ ...
[2023-08-24 03:31:10,432 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xqa/daas%E8%87%AA%E5%8A%A8%E5%8C%96/ ...
[2023-08-24 03:31:10,433 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/xobj/ ...
[2023-08-24 03:31:10,434 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/app/2023/03/maintenance/ ...
[2023-08-24 03:31:10,436 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xqa/xee/some%E5%B8%B8%E8%A7%81%E5%90%8D%E8%AF%8D%E8%A7%A3%E9%87%8A/ ...
[2023-08-24 03:31:10,436 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/2023/02/tool/ ...
[2023-08

[2023-08-24 03:31:10,481 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/process/code-style/protobuf/ ...
[2023-08-24 03:31:10,483 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/xobj/db/v1/ ...
[2023-08-24 03:31:10,485 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xqa/daas%E4%B8%9A%E5%8A%A1%E6%B5%8B%E8%AF%95/%E5%B8%B8%E7%94%A8xdp%E8%B4%A6%E5%8F%B7%E5%AF%86%E7%A0%81/ ...
[2023-08-24 03:31:10,485 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xqa/daas%E4%B8%9A%E5%8A%A1%E6%B5%8B%E8%AF%95/datawand%E6%95%B0%E6%8D%AE%E5%BA%93/ ...
[2023-08-24 03:31:10,486 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xqa/daas%E8%87%AA%E5%8A%A8%E5%8C%96/python-pip%E7%9B%B8%E5%85%B3/ ...
[2023-08-24 03:31:10,490 - 

[2023-08-24 03:31:10,540 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/app/2023/08/how-check-mount/ ...
[2023-08-24 03:31:10,540 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/iodc/2023/07/iodc-app/ ...
[2023-08-24 03:31:10,541 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/xobj/db/ ...
[2023-08-24 03:31:10,542 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/iodc/2023/08/troubleshooting/ ...
[2023-08-24 03:31:10,543 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/2023/02/proto-format-tool/ ...
[2023-08-24 03:31:10,544 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xqa/k8s%E5%AD%A6%E4%B9%A0/k8s%E5%AD%A6%E4%B9%A0%E8%B5%84%

[2023-08-24 03:31:10,597 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/06/sgx/pccs/ ...
[2023-08-24 03:31:10,598 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/06/openenclave/ ...
[2023-08-24 03:31:10,599 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2023/05/xss/laniakea/how-to/csv/ ...
[2023-08-24 03:31:10,600 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/12/sgx/problems/ ...
[2023-08-24 03:31:10,601 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/09/gpu/ ...
[2023-08-24 03:31:10,602 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2023/05/xss/laniakea/architecture/k8s/ ...
[2023-08-24 03:31:10,602 -

[2023-08-24 03:31:10,669 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/06/all_in_one_machine/ ...
[2023-08-24 03:31:10,670 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/09/gpu/papers/ ...
[2023-08-24 03:31:10,671 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2023/05/xss/laniakea/architecture/os/ ...
[2023-08-24 03:31:10,672 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2023/01/xss/pki/refs/configs/ ...
[2023-08-24 03:31:10,673 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2023/05/xss/pki/tutorial/client/cluster/ ...
[2023-08-24 03:31:10,675 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/09/gpu/amd-tmz

[2023-08-24 03:31:10,730 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2023/05/xss/laniakea/features/ ...
[2023-08-24 03:31:10,730 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/06/weekly-meetings/ ...
[2023-08-24 03:31:10,731 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/10/constellation/ ...
[2023-08-24 03:31:10,733 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/10/swtpm/ ...
[2023-08-24 03:31:10,733 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/10/intel-project-amber/ ...
[2023-08-24 03:31:10,734 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xss/2022/05/research/ ...
[2023-08-24 03:31:10,735 - INFO

[2023-08-24 03:31:10,784 - INFO - /tmp/ipykernel_1103941/2157818697.py:29 - bbt-xego] - processing http://xego-dev.basebit.me/doc/xobj/app/2023/07/common-authentication/ ...
save store to /home/laisky/data/langchain/index-azure/bbt-xego


In [6]:
# ====================================
# 生成用于问答的 query chain
# ====================================

from langchain.chains import VectorDBQAWithSourcesChain, RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""Use the following pieces of context to answer the users question.
Take note of the sources and include them in the answer in the format: "SOURCES: source1 source2", use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

index = load_store(
    dirpath=index_dirpath,
    name="bbt-xego",
)

chain_type_kwargs = {"prompt": prompt}

# llm = ChatOpenAI(
#     model_name="gpt-3.5-turbo", 
#     temperature=0, 
#     max_tokens=1000)  

llm = AzureChatOpenAI(
    deployment_name=azure_gpt_deploymentid,
    model_name="gpt-3.5-turbo",
)

# chain = VectorDBQAWithSourcesChain.from_chain_type(
#     llm=llm,
#     vectorstore=index.store,
#     return_source_documents=True,
#     chain_type_kwargs=chain_type_kwargs,
#     reduce_k_below_max_tokens=True,
# )

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.store.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
    reduce_k_below_max_tokens=True,
)



In [7]:
# ====================================
# chatbot as service
# ====================================
question = "如何实现 PKI"
result = chain(
    {
        "question": question,
    },
    return_only_outputs=True,
)

print(f"🤔️: {question}\n")
print(f"🤖️: {pretty_print(result['answer'])}\n")
print(f"📖: {result['sources']}")

🤔️: 如何实现 PKI

🤖️: PKI可以实现一个加解密的数据处理层，可以接入一个S3
    API作为外部存储介质。PKI系统内会存储一个用户表，保存用户名、uid、角色、TOTP token、recov
    erCode，有且仅有一个超管用户，超管可以修改PKI的所有配置。操作PKI，除了要进行SSO，还需要输入TOT
    P密钥证明身份。可以为CA设置ca_admin，CA admin可以启用CA的2-step认证。不使用数据库，所
    有需要持久化的数据都以文件的形式加密后存储到S3。持久化的数据包括：【加密】masterKey【加密】超管账户信
    息（用户名、密码、recoverCode）【加密】服务端完整配置文件【签名】待签CSR信息（包含。PKI以确保i
    n-use安全。服务初次启动进行初始化，创建超管、recoverAdmins、RootCA、MK、RK、RKS、
    RAK。PKI基于RBAC的设计，会为用户分配角色，角色是一组权限的集合，用户通过角色来获得权限。有如下角色：超
    级管理员（superAdmin）证书管理员（certAdmin）恢复管理员（recoverAdmin）用户（us
    er）rootCA必须关联到一个certAdmin，这个certAdmin就是超级管理员。恢复管理员可以将自己的
    职位禅让给其他人。恢复管理员的K和L。

📖: http://xego-dev.basebit.me/doc/xss/2023/02/xss/pki/refs/components/ http://xego-dev.basebit.me/doc/xss/2023/01/xss/pki/refs/flows/ http://xego-dev.basebit.me/doc/xss/2023/01/xss/pki/refs/user-and-perm/
